# 工具需要使用Memory
LLM      Agents    Tools     Memory
- 自定义一个工具来使用LLMChain来总结内容
- 使用read_only_memory来读取内存,共享记忆
- 观察共享记忆和不共享的区别

In [62]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_ollama.chat_models import ChatOllama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory

# 创建LLM
llm = ChatOllama(model="qwen2.5:7b", temperature=0.1)

# 创建PromptTemplate
template = """以下是AI机器人和人类的对话:
{chat_history}
根据输入和上面的记录写一份对话总结。
人类：{input}
"""

prompt = PromptTemplate(
    input_variables=["input", "chat_history"],
    template=template,
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

# 创建一个只读的内存对象
# 这将使得在调用时，内存的内容不会被修改
# 只读内存会在每次调用时返回相同的内容
# 你可以根据需要选择是否使用只读内存
read_only_memory = ReadOnlySharedMemory(
    memory=memory
)

summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=read_only_memory,
)

### 构建工具

In [63]:
# 构建工具
# 搜索工具
from bocha_api import bocha_websearch_tool
search_tool = Tool(
    name="bocha_search",
    func=bocha_websearch_tool,
    description="一个搜索工具，可以用来搜索互联网的信息。使用这个工具来获取最新的信息。",
)

# 总结工具
def summarize_tool(input: str) -> str:
    print("Summarizing...   userdefine Tool")
    print("input_history: ", input)
    summary_chain.run(input)

tools = [ search_tool,
         Tool(
            name="summarize_tool",
            func=summarize_tool,
            description="一个用于对话内容总结的工具。使用这个工具来总结对话内容。",
        )
]

print("tools: ", tools)

tools:  [Tool(name='bocha_search', description='一个搜索工具，可以用来搜索互联网的信息。使用这个工具来获取最新的信息。', func=StructuredTool(name='bocha_websearch_tool', description='使用Bocha Web Search API 进行网页搜索。\n\n参数:\n- query: 搜索关键词\n- freshness: 搜索的时间范围\n- summary: 是否显示文本摘要\n- count: 返回的搜索结果数量\n\n返回:\n- 搜索结果的详细信息，包括网页标题、网页URL、网页摘要、网站名称、网站Icon、网页发布时间等。', args_schema=<class 'langchain_core.utils.pydantic.bocha_websearch_tool'>, func=<function bocha_websearch_tool at 0x10c8b18a0>)), Tool(name='summarize_tool', description='一个用于对话内容总结的工具。使用这个工具来总结对话内容。', func=<function summarize_tool at 0x110720a40>)]


In [64]:
# 创建记忆
memory = ConversationBufferMemory(
  memory_key="chat_history",
  return_messages=True,
)

In [65]:
# 创建Agent
agent_chain = initialize_agent(
  tools=tools,
  llm=llm,
  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
  verbose=True,
  memory=memory,
  handle_parsing_errors=True
)

print(agent_chain.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

bocha_search(tool_input: 'str', callbacks: 'Callbacks' = None) -> 'str' - 一个搜索工具，可以用来搜索互联网的信息。使用这个工具来获取最新的信息。
summarize_tool(input: str) -> str - 一个用于对话内容总结的工具。使用这个工具来总结对话内容。

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [bocha_search, summarize_tool]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [66]:
# 修改模板
prefix = """Answer the following questions as best you can. You have access to the following tools:

bocha_search(tool_input: 'str', callbacks: 'Callbacks' = None) -> 'str' - 一个搜索工具，可以用来搜索互联网的信息。使用这个工具来获取最新的信息。
summarize_tool(input: str) -> str - 一个用于对话内容总结的工具。使用这个工具来总结对话内容。

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [bocha_search, summarize_tool]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
"""

suffix = """
Begin!

History:{chat_history}
Question:{input}
{agent_scratchpad}
"""

# 再把原来的覆盖掉

agent_chain = initialize_agent(
  tools,
  llm,
  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
  verbose=True,
  handle_parsing_errors=True,
  agent_kwargs={
    # "prefix":prefix,
    "suffix":suffix,
    "agent_scratchpad":MessagesPlaceholder("agent_scratchpad"),
    "chat_history":MessagesPlaceholder("chat_history"),
    "input":MessagesPlaceholder("input")
  },
  memory=memory,
)

print(agent_chain.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

bocha_search(tool_input: 'str', callbacks: 'Callbacks' = None) -> 'str' - 一个搜索工具，可以用来搜索互联网的信息。使用这个工具来获取最新的信息。
summarize_tool(input: str) -> str - 一个用于对话内容总结的工具。使用这个工具来总结对话内容。

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [bocha_search, summarize_tool]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


Begin!

History:{chat_history}
Question:{input}
{agent_scratchpad}



In [67]:
agent_chain.run(input="美国第45任总统是谁？")



> Entering new AgentExecutor chain...
Thought: 我需要确定当前最新的信息来回答这个问题，因为美国总统的任期可能会有变动。
Action: bocha_search
Action Input: "美国第45任总统是谁"
Observation: 引用: 1
标题: 唐纳德·特朗普
URL: https://baike.sogou.com/v1919526.htm?fromTitle=%E7%89%B9%E6%9C%97%E6%99%AE
摘要: 编辑词条
添加义项 收藏 分享
唐纳德·特朗普(8)
唐纳德·特朗普（Donald Trump），1946年6月14日出生于美国纽约，国籍美国，毕业于宾夕法尼亚大学，第45、47任美国总统 [1] ，德裔美国共和党籍政治家、企业家。在从政前，曾是商人和电视名人 [2] 。
1968年，获得宾夕法尼亚大学沃顿商学院经济学学士学位，随后任职于父亲弗雷德·特朗普的房地产公司。1971年接管公司，从事房地产开发，投资范围逐步延伸至其他多个行业。2016年11月9日，当选美国第45任总统，2017年1月20日宣誓就职 [3] 。2020年12月，特朗普败选，连任失败 [4] 。2021年1月19日，特朗普发表告别演说 [5] 。特朗普执政期间，特朗普政府在减税、强硬贸易政策、加大外交军事布局、放松金融监管方面通过立法、签署行政令、人事任命等多个方面取得了较大进展，其中税改是其前期政绩的最大亮点 [6] 。2023年3月30日，美国纽约曼哈顿大陪审团投票决定起诉美国前总统特朗普 [7] ；4月4日下午，美国前总统特朗普在纽约曼哈顿地区法院出庭接受问讯 [8] ；4月5日，特朗普正式被拘捕，成为美国历史上第一位受到刑事罪名指控的前总统 [9] 。
美国时间2024年11月6日，特朗普宣布在2024年总统选举中获胜 [10] 。12月17日，特朗普在各州的选举人团会议中获得足够的选举人票，正式获得总统职位 [11] 。当地时间2025年1月6日，美国国会正式确认特朗普当选总统 [12] 。美国东部时间1月20日，特...
网站名称: 搜狗百科
网站图标: https://th.bochaai.com/favicon?domain_url=https://baike.sogou.com/v191

'美国第45任总统是唐纳德·特朗普。他在2016年当选，并在2024年宣布再次当选为美国总统。'

In [68]:
print(agent_chain.memory.buffer)

[HumanMessage(content='美国第45任总统是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='美国第45任总统是唐纳德·特朗普。他在2016年当选，并在2024年宣布再次当选为美国总统。', additional_kwargs={}, response_metadata={})]


In [69]:
agent_chain.run(input="特朗普有几个孩子？")



> Entering new AgentExecutor chain...
Thought: 从之前的对话中，我了解到唐纳德·特朗普是美国第45任总统。为了回答关于他孩子的数量的问题，我需要搜索一下相关信息。
Action: bocha_search
Action Input: "唐纳德·特朗普有几个孩子"
Observation: 引用: 1
标题: 特朗普与三个妻子生育五个子女，非常不一般
URL: https://m.163.com/dy/article/JGCHDQ5O05565QPS.html
摘要: 特朗普与三个妻子生育五个子女，非常不一般：
第一、小唐纳德·特朗普，特朗普大儿子，1977年出生，今年47岁，毕业于宾夕法尼亚大学沃顿商学院，特朗普集团的执行副总裁。
第二、伊万卡·特朗普，特朗普大女儿，1981年出生，今年43岁，毕业于宾夕法尼亚大学沃顿商学院，超级模特，拥有自己的品牌。
第三、埃里克·特朗普，特朗普二儿子，1984年出生，今年40岁，毕业于乔治城大学，特朗普集团的执行副总裁，与哥哥共同管理。
第四、蒂芙尼·特朗普，特朗普女儿，1993年出生，今年31岁，毕业于宾夕法尼亚大学，获得法学博士学位。
第五、巴伦·特朗普，特朗普的小儿子，2006年出生，目前在纽约大学读书。
五个子女，三个是宾夕法尼亚大学毕业，这里是选票的摇摆州，特朗普在宾夕法尼亚州应该是他的福地。
特朗普培养的五个子女都是人中龙凤，儿子管理公司，女儿伊万卡，颜值、情商、名媛都是顶级的，极有可能在四年后会代表共和党进行总统位置角逐，成为美国第一位女总统。
网站名称: 网易
网站图标: https://th.bochaai.com/favicon?domain_url=https://m.163.com/dy/article/JGCHDQ5O05565QPS.html
发布时间: 2024-11-07T07:30:39Z

引用: 2
标题: 特朗普与大女儿从亲亲蜜蜜到产生矛盾分歧
URL: https://m.163.com/dy/article/JOR5CNFK0523TBIA.html
摘要: 特朗普有三个儿子两个女儿，但在这五个孩子中，他最疼爱的孩子莫过于大 女儿 伊万卡  。 伊万卡 一直是父亲特朗普的掌上明珠。伊万卡小时候不论有什么需求，特朗普都会尽可能地满足她。伊

'唐纳德·特朗普共有五个孩子。'

In [70]:
agent_chain.run(input="我需要告诉我孩子我们聊了什么")



> Entering new AgentExecutor chain...
Thought: 我已经了解了之前的对话内容，可以直接总结出对话主题和主要内容。
Final Answer: 你们最近讨论的是关于美国第45任总统唐纳德·特朗普的信息。具体来说，提到了他的任期以及他有五个孩子的事情。

> Finished chain.


'你们最近讨论的是关于美国第45任总统唐纳德·特朗普的信息。具体来说，提到了他的任期以及他有五个孩子的事情。'